In [1]:
import pandas as pd
import numpy as np
import fiona
from utils.loc_utils import *

In [2]:
data_base_path = './data/taxi-data/'
taxi_data_path = data_base_path+'yellow_tripdata_2019-01.csv'
taxi_zones_path = './data/taxi-data/zones/taxi_zones.shp'
save_path = './result/'

# Create Count CSV

In [3]:
taxi_data = pd.read_csv(taxi_data_path)
taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [4]:
# remove unknown places 
pu_cond = (taxi_data['PULocationID']!=263) &(taxi_data['PULocationID']!=264) &(taxi_data['PULocationID']!=265)
do_cond = (taxi_data['DOLocationID']!=263) &(taxi_data['DOLocationID']!=264) &(taxi_data['DOLocationID']!=265)
taxi_data = taxi_data[pu_cond & do_cond]

In [5]:
pu_loc = taxi_data['PULocationID']
do_loc = taxi_data['DOLocationID']

In [6]:
pu_df = create_loc_counts_df(pu_loc,taxi_zones_path)
do_df = create_loc_counts_df(do_loc,taxi_zones_path)
pu_df.head()

/home/rewu/Documents/courses/smart-taxi/utils/loc_utils.py:18: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  poly = shape.next()


,id,latitude,longitude,count
0,237,219247.936518,993760.340994,317817
1,161,215347.285618,990525.078929,306791
2,236,223550.620290,996120.371406,306143
3,162,214897.479832,991834.182817,270248
4,230,216022.344626,988595.728251,260259


In [7]:
pu_save_name = 'pu_count_jan.csv'
do_save_name = 'do_count_jan.csv'

pu_df.to_csv(save_path+pu_save_name,index=False)
do_df.to_csv(save_path+do_save_name,index=False)

#  OD Table 


In [8]:
od = np.zeros((pu_loc.max()+1,do_loc.max()+1))
for pu, do in zip(pu_loc,do_loc):
    od[pu,do] +=1 

In [9]:
od_path = "od_jan.npy"
np.save(save_path+od_path,od)